# Test mpinat_zacros package

## Import packages and get path to Zacros calculations

In [ ]:
import os
import platform 
from pathlib import Path
from nat_zarcos import lattice, state

# Dictionary mapping usernames to data paths
user_paths = {
    'a-DJA'  : Path('c:/Users/a-DJA/Dropbox/Surface_Reaction_Kinetics/O_Pt111/zacros_calculations'),
    'akandra': Path('/home/akandra/O_Pt111/zacros_calculations'),
    # Add more collaborators here as needed
    # note that file names and paths are case-sensitive here, even on Windows because file existence is checked in python
    # in a case sensitive manner
}

if platform.system() == 'Windows':
    username = os.getenv('USERNAME')
else:  # Linux/Mac
    username = os.getenv('USER')

# Get path for current user
if username in user_paths:
    userpath = user_paths[username]
else:
    raise ValueError(f"Unknown user: {username}. Please add your username and path to user_paths dictionary.")

print(f"Operating System : {platform.system()}")
print(f"username         : {username}")
print(f"userpath         : {userpath.as_posix()}")
# Show the type and value
print(f"repr(userpath)   : {repr(userpath)}")

In [ ]:
lat = lattice(userpath / 'fn_3leed/jobs/1/traj_1')
lat.site_coordinations
print( f'len(lat) = {len(lat)}' )

In [ ]:
ads = state(len(lat))
ads.occupation
ads.get_state(userpath / 'fn_3leed/jobs/1/traj_1', idx=0)
ads.occupation